In [1]:
import pandas as pd
from sqlalchemy import create_engine


### Extract CSVs into DataFrames

In [2]:
# import the imdb data
imdb_file = "IMDb movies.csv"
imdb_df = pd.read_csv(imdb_file)

In [3]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81273 entries, 0 to 81272
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_title_id          81273 non-null  object 
 1   title                  81273 non-null  object 
 2   original_title         81273 non-null  object 
 3   year                   81273 non-null  int64  
 4   date_published         81273 non-null  object 
 5   genre                  81273 non-null  object 
 6   duration               81273 non-null  int64  
 7   country                81234 non-null  object 
 8   language               80518 non-null  object 
 9   director               81200 non-null  object 
 10  writer                 79780 non-null  object 
 11  production_company     76948 non-null  object 
 12  actors                 81207 non-null  object 
 13  description            78843 non-null  object 
 14  avg_vote               81273 non-null  float64
 15  vo

In [4]:
# import the netflix data
netflix_file = "netflix_titles.csv"
netflix_df = pd.read_csv(netflix_file)
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6234 entries, 0 to 6233
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       6234 non-null   int64 
 1   type          6234 non-null   object
 2   title         6234 non-null   object
 3   director      4265 non-null   object
 4   cast          5664 non-null   object
 5   country       5758 non-null   object
 6   date_added    6223 non-null   object
 7   release_year  6234 non-null   int64 
 8   rating        6224 non-null   object
 9   duration      6234 non-null   object
 10  listed_in     6234 non-null   object
 11  description   6234 non-null   object
dtypes: int64(2), object(10)
memory usage: 584.6+ KB


In [5]:
#split the netflix data into movies and TV shows
netflix_movies_df = netflix_df.loc[netflix_df['type']=='Movie',:]
netflix_movies_df.info()
netflix_TV_df = netflix_df.loc[netflix_df['type']=='TV Show']
netflix_TV_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4265 entries, 0 to 6231
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       4265 non-null   int64 
 1   type          4265 non-null   object
 2   title         4265 non-null   object
 3   director      4137 non-null   object
 4   cast          3905 non-null   object
 5   country       4070 non-null   object
 6   date_added    4264 non-null   object
 7   release_year  4265 non-null   int64 
 8   rating        4257 non-null   object
 9   duration      4265 non-null   object
 10  listed_in     4265 non-null   object
 11  description   4265 non-null   object
dtypes: int64(2), object(10)
memory usage: 433.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1969 entries, 2 to 6233
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1969 non-null   int64 
 1   type          

In [6]:
# after checking the data we decided to merge on year, director + title
# to avoid mismatches in the data
#check for missing directors in the data and decided to focus on movies
bool_series = pd.isnull(netflix_movies_df["director"])  
print (f"There are {len(netflix_movies_df[bool_series])} movies with no director information") 
bool_series = pd.isnull(netflix_TV_df["director"])  
print (f"There are {len(netflix_TV_df[bool_series])} TV Shows with no director information") 
 

There are 128 movies with no director information
There are 1841 TV Shows with no director information


In [7]:
# merge the two data sets on title, year and director
combined_movie_df = pd.merge(netflix_movies_df, imdb_df, left_on=
['title','director','release_year'], 
right_on=['title','director','year'], suffixes=('_net', '_imdb') )
combined_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1715 entries, 0 to 1714
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   show_id                1715 non-null   int64  
 1   type                   1715 non-null   object 
 2   title                  1715 non-null   object 
 3   director               1715 non-null   object 
 4   cast                   1714 non-null   object 
 5   country_net            1711 non-null   object 
 6   date_added             1715 non-null   object 
 7   release_year           1715 non-null   int64  
 8   rating                 1715 non-null   object 
 9   duration_net           1715 non-null   object 
 10  listed_in              1715 non-null   object 
 11  description_net        1715 non-null   object 
 12  imdb_title_id          1715 non-null   object 
 13  original_title         1715 non-null   object 
 14  year                   1715 non-null   int64  
 15  date

In [8]:
#check for mismatches between the two data sets
#different movies with the same title
combined_movie_df["title"].value_counts()



Benji                        2
The Outsider                 2
One Day                      2
Don                          2
Zoom                         2
                            ..
Saaho                        1
Asees                        1
Tracers                      1
Surga Yang Tak Dirindukan    1
Birdshot                     1
Name: title, Length: 1704, dtype: int64

In [47]:
# check for duplicated imdb id's
duplicateDFRow = combined_movie_df[combined_movie_df["imdb_title_id"].duplicated()]
print(len(duplicateDFRow))
print(duplicateDFRow["imdb_title_id"])

0
Series([], Name: imdb_title_id, dtype: object)


In [9]:
# there are 11 entries that have duplicated titles, and 0 have duplicated id's
pd.options.display.max_rows = 1000 #this allows for full printing
duplicateDFRow = combined_movie_df[combined_movie_df["title"].duplicated()]
print(len(duplicateDFRow))
print(duplicateDFRow["title"])

11
377                   Drive
575            The Outsider
639                  Sarkar
653                   Benji
1053                   Solo
1215    People You May Know
1237            Blood Money
1412                One Day
1419                    Don
1439                   Zoom
1674            The Silence
Name: title, dtype: object


In [10]:
#We drop the duplicated items

combined_movie_df = combined_movie_df.drop_duplicates(subset = ["title"])

combined_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1704 entries, 0 to 1714
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   show_id                1704 non-null   int64  
 1   type                   1704 non-null   object 
 2   title                  1704 non-null   object 
 3   director               1704 non-null   object 
 4   cast                   1703 non-null   object 
 5   country_net            1700 non-null   object 
 6   date_added             1704 non-null   object 
 7   release_year           1704 non-null   int64  
 8   rating                 1704 non-null   object 
 9   duration_net           1704 non-null   object 
 10  listed_in              1704 non-null   object 
 11  description_net        1704 non-null   object 
 12  imdb_title_id          1704 non-null   object 
 13  original_title         1704 non-null   object 
 14  year                   1704 non-null   int64  
 15  date

In [11]:
# cut the columns that are missing a lot of information
#27 budget # 28 usa_gross_income #29 worlwide_gross_income #30 object metascore             
combined_movie_df = combined_movie_df[['show_id', 'type', 'title','director', 'cast', 'country_net', 'date_added', 'release_year', 'rating', 'duration_net',
'listed_in', 'description_net', 'imdb_title_id', 'original_title', 'year',
'date_published', 'genre', 'duration_imdb', 'country_imdb', 'language', 'actors', 'description_imdb', 'avg_vote',
'votes']].copy()
combined_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1704 entries, 0 to 1714
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   show_id           1704 non-null   int64  
 1   type              1704 non-null   object 
 2   title             1704 non-null   object 
 3   director          1704 non-null   object 
 4   cast              1703 non-null   object 
 5   country_net       1700 non-null   object 
 6   date_added        1704 non-null   object 
 7   release_year      1704 non-null   int64  
 8   rating            1704 non-null   object 
 9   duration_net      1704 non-null   object 
 10  listed_in         1704 non-null   object 
 11  description_net   1704 non-null   object 
 12  imdb_title_id     1704 non-null   object 
 13  original_title    1704 non-null   object 
 14  year              1704 non-null   int64  
 15  date_published    1704 non-null   object 
 16  genre             1704 non-null   object 


In [12]:
#Actors and cast are the same information, but cast is missing some data.
#The two durations have the same infomation, but imdb duration is integer
#and netflix is missing some data, so we cut netflix.
#Release year, year and date published are the same, so we cut year and date_published

combined_movie_df = combined_movie_df[['show_id', 'type', 'director', 
'title', 'date_added', 'country_net' ,'release_year', 'rating',
'listed_in', 'description_net', 'imdb_title_id', 'original_title',
'genre', 'duration_imdb',
'country_imdb', 'language', 'actors', 'description_imdb', 'avg_vote',
'votes']].copy()
combined_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1704 entries, 0 to 1714
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   show_id           1704 non-null   int64  
 1   type              1704 non-null   object 
 2   director          1704 non-null   object 
 3   title             1704 non-null   object 
 4   date_added        1704 non-null   object 
 5   country_net       1700 non-null   object 
 6   release_year      1704 non-null   int64  
 7   rating            1704 non-null   object 
 8   listed_in         1704 non-null   object 
 9   description_net   1704 non-null   object 
 10  imdb_title_id     1704 non-null   object 
 11  original_title    1704 non-null   object 
 12  genre             1704 non-null   object 
 13  duration_imdb     1704 non-null   int64  
 14  country_imdb      1703 non-null   object 
 15  language          1693 non-null   object 
 16  actors            1704 non-null   object 


In [13]:
#Country columns are duplicated, and missing information
#We check if we can complete the information by combining the columns
#Check if the country null values are missing in both columns
bool_series = pd.isnull(combined_movie_df['country_net']) 
bool_series = pd.isnull(combined_movie_df['country_imdb'])  
combined_movie_df[bool_series].T  

,132
show_id,80211622
type,Movie
director,Tinge Krishnan
title,Been So Long
date_added,"October 26, 2018"
country_net,NaN
release_year,2018
rating,TV-MA
listed_in,"Dramas, International Movies, Music & Musicals"
description_net,A single mother in London's Camden Town hears ...


In [14]:
#both datasets are missing one piece of data so we replace it with not available
# the imdb database has more country data so we cut country_net
combined_movie_df["country_imdb"].fillna("Not Available", inplace = True)

combined_movie_df = combined_movie_df[['show_id', 'type', 'title', 'director','date_added', 
'release_year', 'rating','listed_in', 'description_net', 'imdb_title_id', 'original_title',
'genre', 'duration_imdb', 'country_imdb', 'language', 'actors', 'description_imdb',
'avg_vote','votes']].copy()
combined_movie_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1704 entries, 0 to 1714
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   show_id           1704 non-null   int64  
 1   type              1704 non-null   object 
 2   title             1704 non-null   object 
 3   director          1704 non-null   object 
 4   date_added        1704 non-null   object 
 5   release_year      1704 non-null   int64  
 6   rating            1704 non-null   object 
 7   listed_in         1704 non-null   object 
 8   description_net   1704 non-null   object 
 9   imdb_title_id     1704 non-null   object 
 10  original_title    1704 non-null   object 
 11  genre             1704 non-null   object 
 12  duration_imdb     1704 non-null   int64  
 13  country_imdb      1704 non-null   object 
 14  language          1693 non-null   object 
 15  actors            1704 non-null   object 
 16  description_imdb  1704 non-null   object 


In [15]:
# fill the missing rating and language data as "not available"
combined_movie_df["rating"].fillna("Not Available", inplace = True)
combined_movie_df["language"].fillna("Not Available", inplace = True)

In [16]:
# check to see if the "original title information is important
different_title = combined_movie_df.title != combined_movie_df.original_title
different_title.sort_values()


0       False
1146    False
1145    False
1144    False
1143    False
        ...  
1714    False
1103     True
664      True
642      True
320      True
Length: 1704, dtype: bool

In [17]:
# manual checking shows that the original title data is not important
# ex: combined_movie_df.iloc[664 , : ]
#cut the original title column
#genre and listed_in information is the same, so cut listed_in
combined_movie_df = combined_movie_df[['show_id', 
'type', 'title', 'director','date_added', 'release_year', 'rating', 'description_net', 'imdb_title_id',
'genre', 'duration_imdb', 'country_imdb', 'language',
'actors', 'description_imdb', 'avg_vote','votes']].copy()
combined_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1704 entries, 0 to 1714
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   show_id           1704 non-null   int64  
 1   type              1704 non-null   object 
 2   title             1704 non-null   object 
 3   director          1704 non-null   object 
 4   date_added        1704 non-null   object 
 5   release_year      1704 non-null   int64  
 6   rating            1704 non-null   object 
 7   description_net   1704 non-null   object 
 8   imdb_title_id     1704 non-null   object 
 9   genre             1704 non-null   object 
 10  duration_imdb     1704 non-null   int64  
 11  country_imdb      1704 non-null   object 
 12  language          1704 non-null   object 
 13  actors            1704 non-null   object 
 14  description_imdb  1704 non-null   object 
 15  avg_vote          1704 non-null   float64
 16  votes             1704 non-null   int64  


In [18]:
combined_movie_df['date_added']= pd.to_datetime(combined_movie_df['date_added'])
combined_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1704 entries, 0 to 1714
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   show_id           1704 non-null   int64         
 1   type              1704 non-null   object        
 2   title             1704 non-null   object        
 3   director          1704 non-null   object        
 4   date_added        1704 non-null   datetime64[ns]
 5   release_year      1704 non-null   int64         
 6   rating            1704 non-null   object        
 7   description_net   1704 non-null   object        
 8   imdb_title_id     1704 non-null   object        
 9   genre             1704 non-null   object        
 10  duration_imdb     1704 non-null   int64         
 11  country_imdb      1704 non-null   object        
 12  language          1704 non-null   object        
 13  actors            1704 non-null   object        
 14  description_imdb  1704 n

In [19]:
#rearrange the columns to place important information first
# remove type as they are all movies
combined_movie_df = combined_movie_df[['show_id', 'imdb_title_id', 'title','genre', 'duration_imdb',  'director', 'actors',
'release_year', 'rating', 'description_net', 'description_imdb', 
 'country_imdb', 'language','date_added','avg_vote','votes']].copy()


In [20]:
#rename the columns
final_movie_df = combined_movie_df.rename(columns={"show_id": "netflix_id", 
    "imdb_title_id": "imdb_id", "duration_imdb":"duration",
    "country_imdb": "country", "language": "film_language"})
final_movie_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1704 entries, 0 to 1714
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   netflix_id        1704 non-null   int64         
 1   imdb_id           1704 non-null   object        
 2   title             1704 non-null   object        
 3   genre             1704 non-null   object        
 4   duration          1704 non-null   int64         
 5   director          1704 non-null   object        
 6   actors            1704 non-null   object        
 7   release_year      1704 non-null   int64         
 8   rating            1704 non-null   object        
 9   description_net   1704 non-null   object        
 10  description_imdb  1704 non-null   object        
 11  country           1704 non-null   object        
 12  film_language     1704 non-null   object        
 13  date_added        1704 non-null   datetime64[ns]
 14  avg_vote          1704 n

In [21]:
#combined_movie_df.to_csv(r'movie_data.csv', index=False)

combined_movie_df.to_csv(r'Movie_Data.csv', index = False)


In [22]:
# if we need to find additional data we can use this package
# pip install imdbpy

In [23]:
rds_connection_string = "postgres:1117@localhost:5432/netflix_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [24]:

engine.table_names()

['netflix']

In [26]:
# commented out to avoid adding duplicates to the datbase
#final_movie_df.to_sql(name='netflix', con=engine, if_exists='append', index=False)

In [63]:
pd.read_sql_query('select * from netflix', con=engine).head()


,id,netflix_id,imdb_id,title,genre,duration,director,actors,release_year,rating,description_net,description_imdb,country,film_language,date_added,avg_vote,votes
0,1,81145628,tt9428190,Norm of the North: King Sized Adventure,"Animation, Adventure, Comedy",90,"Richard Finn, Tim Maltby","Jennifer Cameron, Brian Dobson, Michael Dobson...",2019,TV-PG,Before planning an awesome wedding for his gra...,An ancient Chinese artifact has been stolen by...,"USA, India, South Korea, China",English,2019-09-09,3.2,185
1,2,70304990,tt1361318,Good People,"Action, Crime, Thriller",90,Henrik Ruben Genz,"Sam Spruell, Michael Fox, Diarmaid Murtagh, Fr...",2014,R,A struggling couple can't believe their luck w...,Discovering a stash of cash in their dead tena...,"USA, UK, Denmark, Sweden",English,2017-09-08,5.5,14655
2,3,70299204,tt2917388,Kidnapping Mr. Heineken,"Action, Crime, Drama",95,Daniel Alfredson,"Jim Sturgess, Sam Worthington, Ryan Kwanten, A...",2015,R,"When beer magnate Alfred ""Freddy"" Heineken is ...","The inside story of the planning, execution, r...","Netherlands, Belgium, UK, USA","English, Dutch, German",2017-09-08,6.1,18168
3,4,80057969,tt3774694,Love,"Drama, Romance",135,Gaspar Noé,"Aomi Muyock, Karl Glusman, Klara Kristin, Ugo ...",2015,NR,A man in an unsatisfying marriage recalls the ...,Murphy is an American living in Paris who ente...,"France, Belgium","English, French",2017-09-08,6.1,39385
4,5,80046728,tt2718440,Moonwalkers,"Action, Comedy",107,Antoine Bardou-Jacquet,"Rupert Grint, Ron Perlman, Robert Sheehan, Ste...",2015,R,"A brain-addled war vet, a failing band manager...",After failing to locate the legendary,"France, Belgium",English,2017-09-08,6.1,7963


In [86]:

imdb_file = "IMDb ratings.csv"
imdb_ratings_df = pd.read_csv(imdb_file)

In [87]:
imdb_ratings_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81273 entries, 0 to 81272
Data columns (total 49 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   imdb_title_id              81273 non-null  object 
 1   weighted_average_vote      81273 non-null  float64
 2   total_votes                81273 non-null  int64  
 3   mean_vote                  81273 non-null  float64
 4   median_vote                81273 non-null  float64
 5   votes_10                   81273 non-null  int64  
 6   votes_9                    81273 non-null  int64  
 7   votes_8                    81273 non-null  int64  
 8   votes_7                    81273 non-null  int64  
 9   votes_6                    81273 non-null  int64  
 10  votes_5                    81273 non-null  int64  
 11  votes_4                    81273 non-null  int64  
 12  votes_3                    81273 non-null  int64  
 13  votes_2                    81273 non-null  int

In [88]:
# rename the columns
imdb_ratings_df = imdb_ratings_df.rename(columns={
    "allgenders_0age_avg_vote": "allgenders_age_0_18_avg_vote", 
    "allgenders_0age_votes": "allgenders_age_0_18_votes",
    "allgenders_18age_avg_vote": "allgenders_age_18_29_avg_vote", 
    "allgenders_18age_votes": "allgenders_age_18_29_votes",
    "allgenders_30age_avg_vote": "allgenders_age_30_44_avg_vote", 
    "allgenders_30age_votes": "allgenders_age_30_44_votes",
    "allgenders_45age_avg_vote": "allgenders_age_over_45_avg_vote", 
    "allgenders_45age_votes": "allgenders_age_over_45_votes"
})
    


In [89]:
imdb_ratings_df.head().T

,0,1,2,3,4
imdb_title_id,tt0000574,tt0001892,tt0002101,tt0002130,tt0002199
weighted_average_vote,6.1,5.9,5.2,7,5.7
total_votes,537,171,420,2019,438
mean_vote,6.3,6.1,5.2,6.9,5.8
median_vote,6,6,5,7,6
votes_10,54,5,12,194,28
votes_9,17,6,8,208,15
votes_8,55,17,16,386,42
votes_7,121,41,60,571,75
votes_6,122,52,89,308,114


In [91]:
imdb_ratings_df = imdb_ratings_df.rename(columns={
    "males_0age_avg_vote": "male_age_0_18_avg_vote", 
    "males_0age_votes": "male_age_0_18_votes",
    "males_18age_avg_vote": "male_age_18_29_avg_vote", 
    "males_18age_votes": "male_age_18_29_votes",
    "males_30age_avg_vote": "male_age_30_44_avg_vote", 
    "males_30age_votes": "male_age_30_44_votes",
    "males_45age_avg_vote": "male_age_over_45_avg_vote", 
    "males_45age_votes": "male_age_over_45_votes"
})



In [92]:
imdb_ratings_df = imdb_ratings_df.rename(columns={
    "females_0age_avg_vote": "female_age_0_18_avg_vote", 
    "females_0age_votes": "female_age_0_18_votes",
    "females_18age_avg_vote": "female_age_18_29_avg_vote", 
    "females_18age_votes": "female_age_18_29_votes",
    "females_30age_avg_vote": "female_age_30_44_avg_vote", 
    "females_30age_votes": "female_age_30_44_votes",
    "females_45age_avg_vote": "female_age_over_45_avg_vote", 
    "females_45age_votes": "female_age_over_45_votes"
})



In [93]:
# merge the film data and the ratings data on the imdb id
movie_ratings = pd.merge(final_movie_df, imdb_ratings_df,left_on =
['imdb_id'], right_on = ['imdb_title_id'])


In [94]:
movie_ratings.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1704 entries, 0 to 1703
Data columns (total 65 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   netflix_id                       1704 non-null   int64         
 1   imdb_id                          1704 non-null   object        
 2   title                            1704 non-null   object        
 3   genre                            1704 non-null   object        
 4   duration                         1704 non-null   int64         
 5   director                         1704 non-null   object        
 6   actors                           1704 non-null   object        
 7   release_year                     1704 non-null   int64         
 8   rating                           1704 non-null   object        
 9   description_net                  1704 non-null   object        
 10  description_imdb                 1704 non-null   object     

In [95]:
#drop the columns that are not relevant to ratings
ratings = movie_ratings.drop(columns=['netflix_id','imdb_title_id','genre', 'duration', 'director', 'actors',
'release_year', 'rating', 'description_net', 'description_imdb',
'country', 'film_language', 'date_added', 'avg_vote', 'votes'])


In [96]:
#ratings.head().T

In [97]:
# # We could mark missing columns as Not Available, but they shoud be null in SQL
# final_movie_df.loc[final_movie_df["title"] == "FirstBorn"]

In [98]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1704 entries, 0 to 1703
Data columns (total 50 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   imdb_id                          1704 non-null   object 
 1   title                            1704 non-null   object 
 2   weighted_average_vote            1704 non-null   float64
 3   total_votes                      1704 non-null   int64  
 4   mean_vote                        1704 non-null   float64
 5   median_vote                      1704 non-null   float64
 6   votes_10                         1704 non-null   int64  
 7   votes_9                          1704 non-null   int64  
 8   votes_8                          1704 non-null   int64  
 9   votes_7                          1704 non-null   int64  
 10  votes_6                          1704 non-null   int64  
 11  votes_5                          1704 non-null   int64  
 12  votes_4             

### Transform premise DataFrame

### Transform county DataFrame

### Create database connection

### Load DataFrames into database

In [99]:
rds_connection_string = "postgres:1117@localhost:5432/netflix_db"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

['netflix', 'ratings']

In [100]:
# commented out to avoid adding duplicates to the datbase

#ratings.to_sql(name='ratings', con=engine, if_exists='append', index=False)

In [ ]:
# Is it better to have nulls in the sql data base?
# make a bridge table?
